In [ ]:
import time
import os
import numpy as np
import cv2
import onnxruntime as ort
from buildhat import Motor
from picamera2 import Picamera2

# Motors
motor_C = Motor('C')  # Right
motor_D = Motor('D')  # Left

# Speeds
SCAN_SPEED = 40
MOVE_SPEED = 70

# Model
MODEL_PATH = "/home/aman1234/Downloads/best.onnx"
CONFIDENCE_LOW = 0.3
CONFIDENCE_HIGH = 0.6

# Initialize camera
picam2 = Picamera2()
picam2.preview_configuration.main.size = (640, 480)
picam2.preview_configuration.main.format = "RGB888"
picam2.configure("preview")
picam2.start()
time.sleep(1)

# Load ONNX model
session = ort.InferenceSession(MODEL_PATH)
input_name = session.get_inputs()[0].name

def preprocess(frame):
    resized = cv2.resize(frame, (640, 640))
    rgb = resized[:, :, ::-1]  # BGR to RGB
    norm = rgb.astype(np.float32) / 255.0
    transposed = np.transpose(norm, (2, 0, 1))[np.newaxis, :]
    return transposed

def detect_trash(frame):
    input_tensor = preprocess(frame)
    outputs = session.run(None, {input_name: input_tensor})
    detections = outputs[0][0]

    for det in detections:
        x_c, y_c, w, h, conf = det
        if conf > 0:
            return conf
    return 0  # No detection

def rotate_scan(timeout=8):
    print("Starting 360 scan...")
    start_time = time.time()
    motor_C.start(SCAN_SPEED)
    motor_D.start(-SCAN_SPEED)

    detected_conf = 0
    while time.time() - start_time < timeout:
        frame = picam2.capture_array()
        conf = detect_trash(frame)
        print(f"Detected confidence: {conf:.2f}")
        if conf > CONFIDENCE_LOW:
            detected_conf = conf
            break
        time.sleep(0.3)

    motor_C.stop()
    motor_D.stop()
    return detected_conf > CONFIDENCE_LOW

def move_forward(timeout=10):
    print("Moving forward...")
    motor_C.start(MOVE_SPEED)
    motor_D.start(MOVE_SPEED)

    start_time = time.time()
    while time.time() - start_time < timeout:
        frame = picam2.capture_array()
        conf = detect_trash(frame)
        print(f"While moving - confidence: {conf:.2f}")
        if conf > CONFIDENCE_HIGH:
            print("High confidence trash detected!")
            break
        time.sleep(0.3)

    motor_C.stop()
    motor_D.stop()

def stop_all():
    motor_C.stop()
    motor_D.stop()

def main():
    try:
        while True:
            found = rotate_scan()
            if found:
                move_forward()
            else:
                print("No trash found. Retrying...")
                time.sleep(1)
    except KeyboardInterrupt:
        print("Exiting...")
        stop_all()
        picam2.stop()

if __name__ == "__main__":
    main()
